In [137]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import yfinance as yf

In [138]:
tsla = yf.Ticker("tsla")

tsla_hist = tsla.history(period='5y', interval='1d', end='2025-06-13')
# tsla_hist

In [139]:
tsla_hist.drop(columns=['Dividends', 'Stock Splits'], inplace=True)

In [140]:
tsla_hist.index =pd.to_numeric(tsla_hist.index)
tsla_hist.index = tsla_hist.index/(max(tsla_hist.index)) + 1

In [141]:
for col in tsla_hist.columns:
    tsla_hist[col] = tsla_hist[col]/max(tsla_hist[col]) + 1

In [142]:
X = tsla_hist.drop(columns=['Close'])
y = tsla_hist['Close']

In [143]:
X['Date'] = X.index

In [144]:
for delay in range(1,4):
    X[f"Delay {delay}"] = y.iloc[(3-delay):-(delay)]

In [145]:
X_sample = [[d, h, l, o, vol, v1, v2, v3] for d, h, l, o, vol, v1, v2, v3 in zip(X['Date'].iloc[3:].values, X['High'].iloc[3:].values, X['Low'].iloc[3:].values,X['Open'].iloc[3:].values, X['Volume'].iloc[3:].values, X['Delay 1'].dropna().values, X['Delay 2'].dropna().values, X['Delay 3'].dropna().values)]
y_sample = y.iloc[3:].values

In [146]:
len(X['Date'].iloc[3:].values), len(X_sample)

(1252, 1252)

In [147]:
len(y_sample), len(X_sample)

(1252, 1252)

In [148]:
torch.set_default_dtype(torch.float64)

In [149]:
X = torch.from_numpy(np.array(X))
y = torch.from_numpy(np.array(y))

In [150]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [151]:
train_data = [(X,y) for X, y in zip(X[:-273], y[:-273])]
test_data = [(X,y) for X, y in zip(X[-273:], y[-273:])]

In [152]:
def fill_nan_values(dataset):
    nan_indexes = []
    for tup in range(len(dataset)):
        if (torch.any(dataset[tup][0].isnan(), dim=0)):
            print(f"Found a nan value at index {tup}")
            nan_indexes.append(tup)
    
    for i in nan_indexes:
        temp_tensor = torch.nan_to_num(dataset[i][0], nan=1)
        temp_tuple = (temp_tensor, dataset[i][1])
        dataset[i] = temp_tuple
        print(f"Replaced X & y at index {i}")

In [153]:
fill_nan_values(train_data)
fill_nan_values(test_data)

Found a nan value at index 0
Found a nan value at index 1
Replaced X & y at index 0
Replaced X & y at index 1
Found a nan value at index 270
Found a nan value at index 271
Found a nan value at index 272
Replaced X & y at index 270
Replaced X & y at index 271
Replaced X & y at index 272


In [154]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=32,)
test_dataloader= DataLoader(test_data, batch_size=32,)

In [155]:
count = 0
batch = 0
for (X, y) in train_dataloader:
    if (torch.any(X.isnan())):
        print(f"Found on batch: {batch}")
        print(X)
        print(torch.nan_to_num(X, nan=0))
        count+=1
    batch +=1


In [156]:
class MLP(nn.Module):
    def __init__(self, input_features, hidden_features, output_features):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(input_features, hidden_features),
            nn.ReLU()
        )
        self.l2 = nn.Sequential(
            nn.Linear(hidden_features, 15),
            nn.ReLU()
        )
        self.l3 = nn.Linear(15, output_features)

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x

In [157]:
mlpmodel = MLP(8, 30, 1)

In [158]:
loss_fn = nn.MSELoss()

optimizer = torch.optim.SGD(params=mlpmodel.parameters(), lr=0.01)

In [159]:
EPOCHS = 1

mlpmodel.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch, (X, y) in enumerate(train_dataloader):
        preds = mlpmodel(X)
        # print(f"Preds + {preds}")
        loss = loss_fn(torch.unsqueeze(y, 1), preds)
        total_loss+= loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"We're on batch: {batch} + Loss: {total_loss/batch}")


We're on batch: 0 + Loss: inf
We're on batch: 1 + Loss: 3.2556098491744967
We're on batch: 2 + Loss: 2.300039698112191
We're on batch: 3 + Loss: 1.979728436246897
We're on batch: 4 + Loss: 1.8594580256795221
We're on batch: 5 + Loss: 1.7154895198244577
We're on batch: 6 + Loss: 1.5847503164340353
We're on batch: 7 + Loss: 1.4549818590659531
We're on batch: 8 + Loss: 1.3471308785338068
We're on batch: 9 + Loss: 1.2521085226409736
We're on batch: 10 + Loss: 1.1768695434732925
We're on batch: 11 + Loss: 1.1044505252737313
We're on batch: 12 + Loss: 1.029173405148924
We're on batch: 13 + Loss: 0.9568478275420318
We're on batch: 14 + Loss: 0.8924819364234536
We're on batch: 15 + Loss: 0.8334045721512722
We're on batch: 16 + Loss: 0.7819727761280435
We're on batch: 17 + Loss: 0.7364580619780047
We're on batch: 18 + Loss: 0.6955830510848489
We're on batch: 19 + Loss: 0.6593158604222848
We're on batch: 20 + Loss: 0.6265523235269099
We're on batch: 21 + Loss: 0.5967267717121606
We're on batch: 

In [160]:
with torch.inference_mode():
    mlpmodel.eval()
    for batch, (X, y) in enumerate(test_dataloader):
        preds = mlpmodel(X)
        loss = loss_fn(torch.unsqueeze(y, dim=1), preds)
        print(f"On Batch: {batch} + With Loss: {loss/batch}")

On Batch: 0 + With Loss: inf
On Batch: 1 + With Loss: 0.0012542051954973802
On Batch: 2 + With Loss: 0.0007359000073950883
On Batch: 3 + With Loss: 0.0017022541177891485
On Batch: 4 + With Loss: 0.010694634939882184
On Batch: 5 + With Loss: 0.007655449466019887
On Batch: 6 + With Loss: 0.0007012798090828775
On Batch: 7 + With Loss: 0.001427795442662036
On Batch: 8 + With Loss: 0.002959751687454326
